# Capstone project 1: House prices advanced regression

## Machine Learning - Polynomial Model

In [89]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

In [80]:
# Load test and train data
Xtrain = pd.read_csv('./dummy_data/X_train.csv',index_col = 'Id')
Xtest = pd.read_csv('./dummy_data/X_test.csv',index_col = 'Id')
ytrain = pd.read_csv('./dummy_data/y_train.csv',header=None,index_col=0)
ytest = pd.read_csv('./dummy_data/y_test.csv',header=None,index_col=0)

### Full model

In [23]:
num_cols = Xtrain.columns[:36].drop('SoldTime')
cat_cols = Xtrain.columns[36:]

In [37]:
# transform to polynomial
transPoly = PolynomialFeatures(2)

In [38]:
Xtrain_cat = Xtrain[cat_cols]
Xtrain_num = transPoly.fit_transform(Xtrain[num_cols])
Xtest_cat = Xtest[cat_cols]
Xtest_num = transPoly.transform(Xtest[num_cols])

In [43]:
Xtrain_cat.shape,Xtrain_num.shape

((1022, 283), (1022, 666))

In [44]:
Xtest_cat.shape,Xtest_num.shape

((438, 283), (438, 666))

In [64]:
X_train = [list(Xtrain_cat.iloc[i].values) + list(Xtrain_num[i]) for i in range(1022)]


In [65]:
X_test = [list(Xtest_cat.iloc[i].values) + list(Xtest_num[i]) for i in range(438)]

In [84]:
reg = LinearRegression().fit(X_train, ytrain)

In [85]:
predict = reg.predict(X_test)

In [88]:
predict.shape,ytest.shape

((438, 1), (438, 1))

In [90]:
r2_score(predict,ytest)

-0.0020996226672027607

In [91]:
r2_score(reg.predict(X_train),ytrain)

0.9891409875982647

### Selected features

In [93]:
# According to feature EDA, sellect only the interesting features

interesting_columns = ['RoofMatl', 'Exterior1st','Neighborhood','LotFrontage', 'LotArea','OverallQual', \
                     'OverallCond', 'YearBuilt', 'YearRemodAdd','BsmtFinSF1','ExterCond','BsmtQual',\
                     'HeatingQC','CentralAir','KitchenQual','FireplaceQu','SaleType', 'SaleCondition', \
                     'MSSubClass','TotalBsmtSF', '1stFlrSF','2ndFlrSF','GrLivArea','GarageCars', 'GarageArea']

In [100]:
Features = []

In [101]:
for col in interesting_columns:
    if col in Xtrain.columns:
        Features.append(col)

In [103]:
print(Features)

['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageCars', 'GarageArea']


In [104]:
Xtrain = Xtrain[Features]
Xtest = Xtest[Features]

In [107]:

X_train = transPoly.fit_transform(Xtrain[Features])

X_test = transPoly.transform(Xtest[Features])

In [108]:
reg = LinearRegression().fit(X_train, ytrain)

In [109]:
predict2=reg.predict(X_test)

In [110]:
r2_score(reg.predict(X_train),ytrain),r2_score(predict2,ytest)

(0.8890363162332623, 0.6904278288245461)